In [17]:
import os
import logging
import pandas as pd
import numpy as np
import lightgbm as lgb
from models.data_loader import data_loader, data_loader_all
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [18]:
df = pd.read_csv('input/train/0.csv')
df.head(10)

,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,...,V5111,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120
0,0,30.486806,8.682230,8.736399,8.691251,8.711798,204.576461,201.196818,-1.213967e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,127.58,112.263,-2.647025e-01,85.4,-0.004474
1,1,30.460030,8.784153,8.691244,8.706254,8.716143,194.222395,169.979333,-2.279681e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,1.449708e-05,85.4,0.000000
2,2,30.475103,8.812022,8.697733,8.721917,8.681361,193.078139,160.719360,-1.586760e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-1.297699e-05,85.4,0.000000
3,3,30.455413,8.908323,8.712004,8.723829,8.724655,207.644178,232.675873,-4.016423e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,1.920643e-05,85.4,0.000000
4,4,30.472642,8.815620,8.702882,8.722939,8.702341,186.112451,160.829340,8.676680e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-8.471766e-06,85.4,0.000000
5,5,30.479996,8.785063,8.724755,8.706315,8.679515,179.695790,185.416531,3.727744e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-6.392587e-07,85.4,0.000000
6,6,30.453275,8.750956,8.694296,8.741906,8.732474,181.768368,221.457188,-1.592450e-20,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,3.142552e-05,85.4,0.000000
7,7,30.471123,8.719291,8.662265,8.767139,8.718461,196.023732,158.825020,4.217813e-19,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-4.905278e-06,85.4,0.000000
8,8,30.474419,8.780911,8.722811,8.707110,8.725835,203.839786,203.675179,-2.358491e-20,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,1.097287e-05,85.4,0.000000
9,9,30.480129,8.770054,8.683935,8.755444,8.722334,194.606116,210.825506,-9.246151e-20,0.0,...,1.0,1.0,1.0,1.0,60.0,0.00,0.000,-1.293516e-05,85.4,0.000000


In [21]:
!python test.py -col 'V1150' 'np.mean' 

100%|█████████████████████████████████████████| 100/100 [00:06<00:00, 15.49it/s]
(10000, 5124) (6000, 5123)
1 fold
[   0    5    6 ... 9997 9998 9999]
slicing
making dataset 1
Traceback (most recent call last):
  File "test.py", line 97, in <module>
    early_stopping_rounds=10)
  File "/home/unerue/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py", line 228, in train
    booster = Booster(params=params, train_set=train_set)
  File "/home/unerue/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py", line 1722, in __init__
    train_set.construct().handle,
  File "/home/unerue/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py", line 1087, in construct
    categorical_feature=self.categorical_feature, params=self.params)
  File "/home/unerue/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py", line 889, in _lazy_init
    self.__init_from_np2d(data, params_str, ref_dataset)
  File "/home/unerue/anaconda3/envs/kaggle/lib

In [3]:
train_path = 'input/train'
test_path = 'input/test'
label = pd.read_csv('input/train_label.csv')
sub = pd.read_csv('input/sample_submission.csv')

train = data_loader_all(data_loader, 
                        path=train_path, 
                        train=True, 
                        nrows=100, # 600
                        normal=999, 
                        event_time=10, 
                        lookup_table=label)

test = data_loader_all(data_loader, 
                       path=test_path, 
                       train=False, 
                       nrows=60)

print(train.shape, test.shape)

100%|██████████| 720/720 [00:47<00:00, 15.02it/s]


(82800, 5124) (43200, 5123)


In [ ]:
# train.info()

In [ ]:
# train.to_pickle('train.pkl')
# # test.to_pickle('test.pkl')

In [4]:
object_columns = train.select_dtypes(include=['object']).columns.tolist()
print(object_columns[:5], len(object_columns))

['V0019', 'V0020', 'V0021', 'V0022', 'V0023'] 1026


In [5]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
# train[object_columns] = train[object_columns].replace(
#     ['Equip Fail', 'Bad', 'OFF'], -1)

In [ ]:
# for col in object_columns:
#     train[col] = train[col].astype(np.float64)

In [6]:
train['label'].unique()

array([999, 132,  54, 113,  73, 128, 114,  14,  17,  84,  50,  18, 148,
        58,  23,  38, 180, 183, 115,  69,  29,  91, 117, 112,  61, 173,
       121, 174,  28,  55, 166, 167, 189, 118, 169,  22, 176,  27, 111,
        19, 116,  85, 141,  66,  40,  81,  53,  87, 137, 150,  90, 100,
       175, 165, 177, 160,  76, 127, 130, 123, 136, 162,  80,  68, 181,
        67,  62, 120, 185, 110, 122,  32, 168, 126, 178, 104, 172,  24,
        11, 147,  13,  26, 164, 152,   8,  21, 157, 192,  83, 146,   6,
       143, 196, 119,  52, 106,  98, 188, 161, 133,  30, 129,  12,  75,
         2,  82,   5,  77,   4,  88, 131,  64,  25, 107,  78, 159, 138,
       105, 154,  89,  96,  39,  60,  10,  65,  94,  47,  63, 163, 179,
         9, 187, 102,  97,  79, 158, 191,  70, 125,  74, 109,  71,  45,
       193,  86, 108,  33,   3,  59,   1, 186,  99,  41,   7, 170, 103,
        57,  56,   0, 139, 149, 140, 197, 155,  20, 124,  42,  16,  35,
        92, 156,  46,  36, 142, 101,  37,  93, 182, 194, 134,  1

In [7]:
labels = dict(zip(train['label'].unique().tolist(), list(range(train['label'].nunique()))))

In [8]:
X = train.drop(['label'] + object_columns, axis=1).values
y = train['label'].replace(labels).values
# y = train['label'].values
test = test.drop(object_columns, axis=1).values

In [9]:
test = np.where(test == 'Bad', 0, test)
test = np.where(test == 'No Data', 0, test)
test = np.where(test == 'Configure', 0, test)
test = np.where(test == 'OFF', 0, test)
test = np.where(test == 'Equip Fail', 0, test)

In [10]:
n_class = train['label'].nunique()
print(n_class)

199


In [11]:
params = {
    'boosting_type': 'gbdt',
    'device': 'gpu',
    'objective': 'multiclass',
    'num_class': n_class,
    'metric': 'multi_logloss',
#     'learning_rate': 0.015,
#     'num_leaves': 255,  
    'max_depth': -1,  
    'max_bin': 50,
#     'min_child_samples': 100,  
#     'max_bin': 100,  
#     'subsample': 0.7,  
#     'subsample_freq': 1,  
#     'colsample_bytree': 0.7,  
#     'min_child_weight': 0,  
#     'subsample_for_bin': 200000,  
#     'min_split_gain': 0,  
#     'reg_alpha': 0,  
#     'reg_lambda': 0,  
   'nthread': -1,
    'verbosity': -1,
#     'scale_pos_weight':99 
    }

In [12]:
train_preds = np.zeros((len(X), n_class))
test_preds = np.zeros((len(test), n_class))
cv_score = []
best_trees = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    X_train_data = X[train_index]
    X_valid_data = X[valid_index]
    y_train_data = y[train_index]
    y_valid_data = y[valid_index]
    
    dtrain = lgb.Dataset(X_train_data, y_train_data)
    dvalid = lgb.Dataset(X_valid_data, y_valid_data)
    
    bst = lgb.train(params, 
                     dtrain, 
                     num_boost_round=100, 
                     valid_sets=[dtrain, dvalid], 
                     verbose_eval=100,
                     early_stopping_rounds=20)
    
    best_trees.append(bst.best_iteration)
    preds = bst.predict(X_valid_data)
    
    train_preds[valid_index] = preds
    test_preds += bst.predict(test, num_iteration=bst.best_iteration) / skf.n_splits
    
#     clf.fit(X_train_data, y_train_data)
    score = log_loss(y_valid_data, preds)
    print('{}-fold: logloss = {})'.format(i+1, score))
    
    cv_score.append(score)
#     train_preds[valid_index] = clf.predict_proba(X_valid_data)
#     test_preds += clf.predict_proba(test.values) / skf.n_splits

# print('\ntrain accuracy score = {:.3}'.format(accuracy_score(y, np.argmax(train_preds, axis=1))))
# print('train f1 score = {}'.format(f1_score(y, np.argmax(train_preds, axis=1), average=None)))

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's multi_logloss: 1.01583	valid_1's multi_logloss: 1.24491
1-fold: logloss = 1.2449148761808344)
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's multi_logloss: 1.00215	valid_1's multi_logloss: 1.22793
2-fold: logloss = 1.2279294168298491)
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's multi_logloss: 1.01934	valid_1's multi_logloss: 1.26454
3-fold: logloss = 1.264542148062215)
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's multi_logloss: 1.01005	valid_1's multi_logloss: 1.30108
4-fold: logloss = 1.301082347841828)
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	training's multi_logloss: 0.989514	valid_1's multi_logloss: 1.21052
5-fold: loglos

In [13]:
preds.shape

(16560, 199)

In [14]:
reversed_labels = dict((v, k) for k, v in labels.items())

In [16]:
# sub.iloc[:, 1:] = test_preds[:,1:]

In [17]:
sub = pd.read_csv('input/sample_submission.csv')

In [18]:
sub.head()

,id,0,1,2,3,4,5,6,7,8,...,188,189,190,191,192,193,194,195,196,197
0,828,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,829,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,830,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,831,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,832,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
sub.iloc[:, 1:].shape

(720, 198)

In [20]:
test_df = pd.DataFrame(test)

In [21]:
test_df1 = test_df.iloc[:,:1]

In [22]:
test_df1 = test_df1.rename(columns={0: 'id'})

In [ ]:
# test_df1 = test_df.sort_values(by=0).iloc[:,:1]

In [ ]:
# test_df1 = test_df1.reset_index(drop=True)

In [23]:
test_pred_df = pd.DataFrame(test_preds[:,1:])

In [24]:
sub1 = pd.concat([test_df1, test_pred_df], axis=1)

In [25]:
sub2 = sub1.groupby(by='id').sum() / sub1.groupby(by='id').count()

In [26]:
sub.iloc[:,1:] = sub2.values

In [27]:
sub.to_csv('submission.csv', index=False)

In [16]:
495000

(495, 0)

In [ ]:
input -> preprossecing -> transform -> learning -> result 0.90